In [5]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np
from copy import deepcopy

NUMBER_OF_COUNTIES = 3143

In [6]:
def demand(x, P, totalPop):
    '''
    Calculates demand as a function of price.

    Args:
        x: Total number of stores in given county (cvxpy.Variable).
        P: Price (cvxpy.Variable or cvxpy.Parameter).
        totalPop: Total population in county (cvxpy.Parameter or numeric).

    Returns:
        demand: Demand corresponding to input variables (cvxpy.Expression).
    '''
    P2 = deepcopy(P)
    for i in range(len(P2)):
        P2[i] = pyo.exp(-0.03 * P[i]**2)  # Coefficient for price
    # L: Maximum demand at unit price 1.0.
    
    L = deepcopy(x)
    for i in range(len(L)):
        L[i] = totalPop / x[i]  # Equivalent to totalPop / x
    # L = totalPop / x  # Equivalent to totalPop / x

    result = deepcopy(P2)
    for i in range(len(result)):
        result[i] = L[i] * P2[i]  # Coefficient for price
    return result

In [3]:
def revenue(P, IR, demand):
    '''
    Calculates Revenue of Product.

    Args:
        P: Price of a given product.
        IR: Income ratio of county to maximum county income (0 to 1).
        demand: The demand of a given product.

    Returns:
        revenue: Demand times Price.
    '''
    result = P.copy()
    for i in range(len(result)):
        result[i] = P[i]**(pyo.sqrt(IR[i])) * demand[i]  # Coefficient for price
    return result

In [4]:
def loss(P, IR, demand):
    '''
    Calculates losses: 8.2% of revenue incurred by restaurant.

    Args:
        P: Price of restaurant.
        IR: Income ratio of county to maximum county income (0 to 1).
        demand: Demand of restaurant.

    Returns:
        loss: Losses incurred by restaurant.
    '''
    return 0.082 * revenue(P, IR, demand)

def costs(P, IR, minwage, rent, demand, employees=15):
    '''
    Calculates costs of restaurant.

    Args:
        minwage: Minimum wage in county.
        rent: Rent of restaurant.
        demand: Demand of restaurant.
        P: Price of restaurant.
        IR: Income ratio of county to maximum county income (0 to 1).
        employees: Number of employees in restaurant (default is 15).

    Returns:
        netCosts: Total costs of restaurant.
    '''
    loss_incurred = loss(P, IR, demand)
    operating_costs = minwage * employees + rent + loss_incurred
    return operating_costs

In [5]:
def newObjectiveFunction(x, P, totalPop, IR, minwage, rent, employees=15):
    '''
    Objective function for optimization problem.

    Args:
        x: Total number of stores + Price per store.
        totalPop: Total population in county.
        IR: Income Ratio relative to maximum county income.
        minwage: Minimum wage in county.
        rent: Rent of restaurant.

    Returns:
        Objective value to minimize.
    '''
    penalty_factor = 1e10  # A large penalty
    penalty = penalty_factor * (P < 10)

    cost_minus_loss = minwage * employees + rent
    
    revenue_x = totalPop * np.exp(-0.03 * P * P) * np.power(P, np.sqrt(IR))

    formula = 0.918 * revenue_x - x * cost_minus_loss
    formula_convex = formula + penalty
    return np.sum(formula_convex)

In [ ]:
def pyomo_optimize(budget, N, risk, totalPop, IR, minwage, rent):
    '''
    Optimizes the objective function given certain variables.

    Args:
        totalPop: Total population in county.
        IR: Income Ratio relative to maximum county income.
        budget (int): Total budget for owning restaurants.
        risk (float): Total acceptable risk ratio per location (cost/revenue).
        N (int): Maximum number of stores to open.
        minwage: Minimum wage in county.
        rent: Rent of restaurant.

    Returns:
        result: Result of optimization function.
    '''

    model = pyo.ConcreteModel()
    model.x = pyo.Var(range(NUMBER_OF_COUNTIES), domain=pyo.NonNegativeReals, name='x') # Number of stores in each county
    model.P = pyo.Var(range(NUMBER_OF_COUNTIES), domain=pyo.NonNegativeReals, name='P') # Price in each county


    # ######################################################
    # Inequality constraints
    # ######################################################

    model.c = pyo.ConstraintList()
    demand_val = demand(model.x, model.P, totalPop)
    cost = costs(model.P,IR,minwage,rent,demand_val)
    total_cost = model.x.T @ cost # Get total costs for budget constraint
    rev = revenue(model.P, IR, demand_val) # Get revenue for risk constraint

    model.c.add(budget - total_cost >= 0) # Budget constraint
    model.c.add(N - sum(model.x) >= 0) # Maximum number of stores constraint
    model.c.add(risk - cost/rev >= 0) # Risk constraint
    model.c.add(model.x >= 0) # Number of stores must be positive
    model.c.add(model.P >= 10) # Price must be positive
    model.c.add(model.P <= 50) # Price must be positive

    model.o = pyo.Objective(
        newObjectiveFunction(model.x, model.P, totalPop, IR, minwage, rent),
        sense=pyo.maximize
    ) # Get objective function
    
    # solver = pyo.SolverFactory('ipopt')
    solver = pyo.SolverFactory('cbc')
    result = solver.solve(model, tee=True) # Solve the model
    return model.x, model.P, result


In [7]:
budget = 10000000
N = 4000
risk = 0.8

print(budget, N, risk)

from readData import *
from rent_estimation import calculate_rent_estimation

income = getIncome()
income.drop(['MedianIncome','MedianIncomeRatio'],axis=1,inplace=True)
populations = getPopulation()
populations = populations[['State','County','2024']]
populations.rename({'2024':'Population'},axis=1,inplace=True)
rent = calculate_rent_estimation()
rent = rent[['State','County','Estimated_annual_rent']]
minwage = getMinWage()

data = populations.merge(income)
data = data.merge(rent)
data = data.merge(minwage)
data

10000000 4000 0.8
Average Rent Per Sqft: 92.93
Average Annual Rent: 185857.18


c:\MY FILES\UMD\Sem 4\MSML604 Optimization\Project\optimization-location-intelligence\rent_estimation.py:110: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['Commercial_rent_per_sqft_year'].fillna(avg_rent_per_sqft, inplace=True)
c:\MY FILES\UMD\Sem 4\MSML604 Optimization\Project\optimization-location-intelligence\rent_estimation.py:111: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace 

,State,County,Population,CountyID,MeanIncome,MeanIncomeRatio,Estimated_annual_rent,MinWage
0,Alabama,Autauga,61464,0500000US01001,93367,0.411925,184682.27,7.25
1,Alabama,Baldwin,261608,0500000US01003,100105,0.441653,282247.68,7.25
2,Alabama,Barbour,24358,0500000US01005,64745,0.285648,106381.32,7.25
3,Alabama,Bibb,22258,0500000US01007,67735,0.298840,146669.94,7.25
4,Alabama,Blount,60163,0500000US01009,79203,0.349435,168697.13,7.25
...,...,...,...,...,...,...,...,...
3138,Wyoming,Sweetwater,41273,0500000US56037,98608,0.435048,211876.05,7.25
3139,Wyoming,Teton,23272,0500000US56039,185173,0.816964,1621554.10,7.25
3140,Wyoming,Uinta,20621,0500000US56041,89995,0.397048,235265.57,7.25
3141,Wyoming,Washakie,7662,0500000US56043,80444,0.354910,184506.77,7.25


In [19]:
results = pyomo_optimize(budget = budget,
                   N = N,
                   risk = risk,
                   totalPop = data['Population'],
                   IR = data['MeanIncomeRatio'],
                   minwage = data['MinWage'],
                   rent = data['Estimated_annual_rent'])


TypeError: unsupported operand type(s) for *: 'float' and 'method'

In [9]:
model = pyo.ConcreteModel()
model.a = pyo.Var(range(5), name='test')
model.b = pyo.Var(range(5), name='test2')

# from pyomo.core.util import prod
# prod([model.a, model.b])  # This will give you the product of all elements in model.a

demand(model.a, model.b, model.b)

ERROR: evaluating object as numeric value: b[0]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized NumericValue object b[0]
ERROR: evaluating object as numeric value: exp(-0.03*b[0]**2)
        (object: <class
        'pyomo.core.expr.numeric_expr.UnaryFunctionExpression'>)
    No value for uninitialized NumericValue object b[0]


ValueError: No value for uninitialized NumericValue object b[0]

In [ ]:
model.a = pyo.Var(range(5), name='test')
model.b = pyo.Var(range(5), name='test2')

In [10]:
model.e = pyo.Var()
model.e**2

In [38]:
model.d = pyo.Set(initialize=[1,2,3])
model.e = pyo.Var(model.d) 

'pyomo.core.base.set.OrderedScalarSet'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.set.AbstractOrderedScalarSet'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().


In [39]:
len(model.a)

5